In [ ]:
#pull all 12 months data
#using FordGO from the SF Bay area and includes 5 cities

In [1]:
import zipfile, urllib.request, shutil
import pandas as pd,numpy as np
import glob
import citipy
import csv
import os
import openweathermapy
from uszipcode import Zipcode

In [2]:
#id the folder that has all the zip files
#create a glob for the path and file names
#create a empty list for holding everything 
path = r'C:\Users\Gogster\Documents\GitHub\RawData'
#path=r'../RawData'
allfiles=glob.glob(path+"/*.zip")

data_list=[]


In [ ]:
# zip_list=[201801-fordgobike-tripdata.csv.zip,
#           201802-fordgobike-tripdata.csv.zip,
#           201803-fordgobike-tripdata.csv.zip,
#           201804-fordgobike-tripdata.csv.zip,
#          201805-fordgobike-tripdata.csv.zip,
#          201806-fordgobike-tripdata.csv.zip,
#          201807-fordgobike-tripdata.csv.zip,
#          201808-fordgobike-tripdata.csv.zip,
#          201809-fordgobike-tripdata.csv.zip,
#          2018010-fordgobike-tripdata.csv.zip,
#          2018011-fordgobike-tripdata.csv.zip,
#          2018012-fordgobike-tripdata.csv.zip]

In [3]:
#list comprehend through the folder with all the zip files
#append to the dataframe merging all the months

for z_file in allfiles:
    df=pd.read_csv(z_file, compression='zip',header=0,sep=',',quotechar='"')
    data_list.append(df)

all_frame=pd.concat(data_list,axis=0,ignore_index=True)    

In [ ]:
all_frame.head()

In [ ]:
all_frame.shape
#this is all the rows 1.863721M by 16

In [4]:
#create 2 new rows 1 for the zip code and one for the city name
#city 
all_frame.insert(loc=7, column='city', value='abc')
#zip codes
all_frame.insert(loc=8, column='zip_code', value='123456')
#age in years of users
all_frame.insert(loc=16, column='age', value ='99')

In [ ]:
#display new table
all_frame.head()

In [5]:
#remove rows which have Nan values
drop_frame= all_frame.dropna(axis=0,how='any',inplace=False)
#delete extra columns in the data that are not relavent
drop_frame.drop(['bike_share_for_all_trip'],axis = 1, inplace = True)
#display the dataframe
drop_frame.head()

C:\Users\Gogster\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,city,zip_code,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,age,member_gender
0,75284,2018-01-31 22:52:35.2390,2018-02-01 19:47:19.8240,120.0,Mission Dolores Park,37.761420,-122.426435,abc,123456,285.0,Webster St at O'Farrell St,37.783521,-122.431158,2765,Subscriber,1986.0,99,Male
2,71576,2018-01-31 14:23:55.8890,2018-02-01 10:16:52.1160,304.0,Jackson St at 5th St,37.348759,-121.894798,abc,123456,296.0,5th St at Virginia St,37.325998,-121.877120,3039,Customer,1996.0,99,Male
4,39966,2018-01-31 19:52:24.6670,2018-02-01 06:58:31.0530,74.0,Laguna St at Hayes St,37.776435,-122.426244,abc,123456,19.0,Post St at Kearny St,37.788975,-122.403452,617,Subscriber,1991.0,99,Male
6,453,2018-01-31 23:53:53.6320,2018-02-01 00:01:26.8050,110.0,17th & Folsom Street Park (17th St at Folsom St),37.763708,-122.415204,abc,123456,134.0,Valencia St at 24th St,37.752428,-122.420628,3571,Subscriber,1988.0,99,Male
7,180,2018-01-31 23:52:09.9030,2018-01-31 23:55:10.8070,81.0,Berry St at 4th St,37.775880,-122.393170,abc,123456,93.0,4th St at Mission Bay Blvd S,37.770407,-122.391198,1403,Subscriber,1980.0,99,Male


In [6]:
drop_frame.shape
# 1.741556 M and 18 col WAS

(1741556, 18)

In [7]:
clean_df=pd.DataFrame(drop_frame)

In [8]:
#find age of user by subtracting birth year from current year
thisyear=2019
clean_df['age'] = thisyear - clean_df['member_birth_year']
clean_df['age'] = clean_df['age'].astype(int)
clean_df['member_birth_year'] = clean_df['member_birth_year'].astype(int)
clean_df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,city,zip_code,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,age,member_gender
0,75284,2018-01-31 22:52:35.2390,2018-02-01 19:47:19.8240,120.0,Mission Dolores Park,37.761420,-122.426435,abc,123456,285.0,Webster St at O'Farrell St,37.783521,-122.431158,2765,Subscriber,1986,33,Male
2,71576,2018-01-31 14:23:55.8890,2018-02-01 10:16:52.1160,304.0,Jackson St at 5th St,37.348759,-121.894798,abc,123456,296.0,5th St at Virginia St,37.325998,-121.877120,3039,Customer,1996,23,Male
4,39966,2018-01-31 19:52:24.6670,2018-02-01 06:58:31.0530,74.0,Laguna St at Hayes St,37.776435,-122.426244,abc,123456,19.0,Post St at Kearny St,37.788975,-122.403452,617,Subscriber,1991,28,Male
6,453,2018-01-31 23:53:53.6320,2018-02-01 00:01:26.8050,110.0,17th & Folsom Street Park (17th St at Folsom St),37.763708,-122.415204,abc,123456,134.0,Valencia St at 24th St,37.752428,-122.420628,3571,Subscriber,1988,31,Male
7,180,2018-01-31 23:52:09.9030,2018-01-31 23:55:10.8070,81.0,Berry St at 4th St,37.775880,-122.393170,abc,123456,93.0,4th St at Mission Bay Blvd S,37.770407,-122.391198,1403,Subscriber,1980,39,Male


In [9]:
max_age = 65
# under65_df = drop_frame.loc[:, (drop_frame.isnull().sum(axis=0) <= max_age)]
# under65_df.shape
#did not work


#Use this DF for all work- It holds all the users aged 19 to 65
max_age_df = (drop_frame[drop_frame["age"]<(max_age)])
max_age_df.shape
#1.71882M records

(1718882, 18)

In [ ]:
#clean_df.to_csv("clean_bike_data.csv")

In [ ]:
#interate through the df and pull out the lat/lng to plug into function search.by_coordinates

In [ ]:
#zip code vs number of stations

In [10]:
#which users are consumers vs subscribers (user_type=Subscriber)
s_grp_cnt= clean_df.groupby('user_type').size()
print (s_grp_cnt)
# 1.741556 M and 18 col

user_type
Customer       199189
Subscriber    1542367
dtype: int64


In [11]:
#group by the sex of the users
# the cnt is correct here
m_grp_cnt= clean_df.groupby('member_gender').size()
print (m_grp_cnt)

member_gender
Female     435010
Male      1279545
Other       27001
dtype: int64


In [12]:
# drop_frame.query(float('member_birth_year') > float('1989.0') and user_type == 'Subsciber').groupby('Sex').count()
#calc the number of customers and subscribers by gender
#WRONG COUNT
cus_df = clean_df.groupby(['user_type','member_gender']).size()
cus_df.head()
#WRONG COUNT

user_type   member_gender
Customer    Female             63193
            Male              131942
            Other               4054
Subscriber  Female            371817
            Male             1147603
dtype: int64

In [13]:
other_df= clean_df.loc[clean_df['member_gender']=='Other']
other_df.head()
# other_df.shape

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,city,zip_code,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,age,member_gender
93,465,2018-01-31 21:56:17.6330,2018-01-31 22:04:02.8280,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,abc,123456,60.0,8th St at Ringold St,37.774520,-122.409449,1168,Subscriber,1911,108,Other
235,2057,2018-01-31 20:21:46.4380,2018-01-31 20:56:04.0060,230.0,14th St at Mandela Pkwy,37.810743,-122.291415,abc,123456,243.0,Bancroft Way at College Ave,37.869360,-122.254337,272,Subscriber,1997,22,Other
242,735,2018-01-31 20:41:04.1420,2018-01-31 20:53:19.6450,6.0,The Embarcadero at Sansome St,37.804770,-122.403234,abc,123456,30.0,San Francisco Caltrain (Townsend St at 4th St),37.776598,-122.395282,1659,Subscriber,1983,36,Other
255,1261,2018-01-31 20:28:01.1690,2018-01-31 20:49:03.0620,186.0,Lakeside Dr at 14th St,37.801319,-122.262642,abc,123456,196.0,Grand Ave at Perkins St,37.808894,-122.256460,1036,Customer,1987,32,Other
396,387,2018-01-31 20:00:54.0730,2018-01-31 20:07:21.5560,19.0,Post St at Kearny St,37.788975,-122.403452,abc,123456,321.0,5th at Folsom,37.780146,-122.403071,2688,Subscriber,1979,40,Other


In [14]:
other_df.shape

(27001, 18)

In [15]:
male_df= clean_df.loc[clean_df['member_gender']=='Male']
male_df.head()
male_df.shape

(1279545, 18)

In [16]:
female_df= clean_df.loc[clean_df['member_gender']=='Female']
female_df.head()
female_df.shape

(435010, 18)

In [18]:
sub_df= clean_df.loc[clean_df['user_type']=='Subscriber']
#sub_df.head()

In [19]:
sub_df.shape
#subscibers only df is 1.542367M by 18

(1542367, 18)

In [20]:
m_sub_df= sub_df.loc[clean_df['member_gender']=='Male']
m_sub_df.shape
#1147603 by 18

(1147603, 18)

In [21]:
f_sub_df= sub_df.loc[clean_df['member_gender']=='Female']
f_sub_df.shape
#371817 by 18

(371817, 18)

In [22]:
o_sub_df= sub_df.loc[clean_df['member_gender']=='Other']
o_sub_df.shape
#cnt is 22947 by 18

(22947, 18)

In [23]:
clean_df['age'].describe()


count    1.741556e+06
mean     3.593921e+01
std      1.045133e+01
min      1.900000e+01
25%      2.800000e+01
50%      3.400000e+01
75%      4.100000e+01
max      1.380000e+02
Name: age, dtype: float64

In [29]:
low_df = clean_df[(clean_df['age']<20)]
low_df.shape

(5289, 18)

In [31]:
o_low_df= low_df.loc[clean_df['member_gender']=='Other']
o_low_df.shape

(43, 18)

In [32]:
m_low_df= low_df.loc[clean_df['member_gender']=='Male']
m_low_df.shape

(4044, 18)

In [33]:
f_low_df= low_df.loc[clean_df['member_gender']=='Female']
f_low_df.shape

(1202, 18)

In [26]:
und23_df = clean_df[(clean_df['age']<23)]
und23_df.shape

(64215, 18)

In [34]:
o_u23_df= und23_df.loc[clean_df['member_gender']=='Other']
o_u23_df.shape

(1454, 18)

In [35]:
m_u23_df= und23_df.loc[clean_df['member_gender']=='Male']
m_u23_df.shape

(46697, 18)

In [36]:
f_u23_df= und23_df.loc[clean_df['member_gender']=='Female']
f_u23_df.shape

(16064, 18)

In [24]:
# # grp all that are under age 37, the millenial? 23 to 37
# age_df= clean_df.loc[clean_df['age']>'23'and ['age']<'37' ]
# age_df.shape

# bins = [20, 25, 30, 35, 37, 40, 45]
# names = ['<20', '20-24','25-29', '30-35', '36 and 37','37-40', '40-44','>45' ]

#fig out millenials 23 to 37
# under23_df= sub_df.loc[clean_df['age'] <'23']
# under23_dfshape

# df[(df[col]>22) & (df[col]<38]

early_mil_df = clean_df[(clean_df['age']>22) & (clean_df['age']<30)]
early_mil_df.shape
#mil_df.head()

(452455, 18)

In [38]:
o_early_mil_df= early_mil_df.loc[clean_df['member_gender']=='Other']
o_early_mil_df.shape

(5123, 18)

In [39]:
m_early_mil_df= early_mil_df.loc[clean_df['member_gender']=='Male']
m_early_mil_df.shape

(318851, 18)

In [40]:
f_early_mil_df= early_mil_df.loc[clean_df['member_gender']=='Female']
f_early_mil_df.shape

(128481, 18)

In [25]:
late_mil_df = clean_df[(clean_df['age']>29) & (clean_df['age']<38)]
late_mil_df.shape

(624847, 18)

In [41]:
o_late_mil_df= late_mil_df.loc[clean_df['member_gender']=='Other']
o_late_mil_df.shape

(9584, 18)

In [42]:
m_late_mil_df= late_mil_df.loc[clean_df['member_gender']=='Male']
m_late_mil_df.shape

(452787, 18)

In [43]:
f_late_mil_df= late_mil_df.loc[clean_df['member_gender']=='Female']
f_late_mil_df.shape

(162476, 18)

In [27]:
to41_df= clean_df[(clean_df['age']>37) & (clean_df['age']<42)]
to41_df.shape

(172961, 18)

In [44]:
o_to41_df= to41_df.loc[clean_df['member_gender']=='Other']
o_to41_df.shape

(4100, 18)

In [45]:
m_to41_df= to41_df.loc[clean_df['member_gender']=='Male']
m_to41_df.shape

(128910, 18)

In [46]:
f_to41_df= to41_df.loc[clean_df['member_gender']=='Female']
f_to41_df.shape

(39951, 18)

In [28]:
ove41_df = clean_df[(clean_df['age']>65)]
ove41_df.shape

(18423, 18)

In [47]:
o_ove41_df= ove41_df.loc[clean_df['member_gender']=='Other']
o_ove41_df.shape

(463, 18)

In [48]:
m_ove41_df= ove41_df.loc[clean_df['member_gender']=='Male']
m_ove41_df.shape

(15766, 18)

In [49]:
f_ove41_df= ove41_df.loc[clean_df['member_gender']=='Female']
f_ove41_df.shape

(2194, 18)

In [ ]:
# create the lat/long and make sure they are in the city limits

In [ ]:
# map the lat/lng over a map of the city

#look at the number of rides over the year

In [ ]:
# id the ride length and then calc the 2sigma

In [ ]:
do we keep these or are the ok